In [1]:
from numpy.polynomial import Polynomial
import numpy as np

In [1]:
"""Example of CKKS multiplication."""

from ckks.ckks_decryptor import CKKSDecryptor
from ckks.ckks_encoder import CKKSEncoder
from ckks.ckks_encryptor import CKKSEncryptor
from ckks.ckks_evaluator import CKKSEvaluator
from ckks.ckks_key_generator import CKKSKeyGenerator
from ckks.ckks_parameters import CKKSParameters


poly_degree = 8
ciph_modulus = 1 << 600
big_modulus = 1 << 1200
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)

key_generator = CKKSKeyGenerator(params)

public_key = key_generator.public_key
secret_key = key_generator.secret_key
relin_key = key_generator.relin_key

encoder = CKKSEncoder(params)
encryptor = CKKSEncryptor(params, public_key, secret_key)
decryptor = CKKSDecryptor(params, secret_key)
evaluator = CKKSEvaluator(params)

message1 = [0.5, 0.3 + 0.2j, 0.78, 0.88j]
message2 = [0.2, 0.11, 0.4 + 0.67j, 0.9 + 0.99j]

plain1 = encoder.encode(message1, scaling_factor)
plain2 = encoder.encode(message2, scaling_factor)
ciph1 = encryptor.encrypt(plain1)
ciph2 = encryptor.encrypt(plain2)
ciph_prod = evaluator.multiply(ciph1, ciph2, relin_key)
# So how define arbitrary operation on the ring
decrypted_prod = decryptor.decrypt(ciph_prod)
decoded_prod = encoder.decode(decrypted_prod)

print(decoded_prod)


[(0.10000000043961196-2.139348254104334e-09j), (0.0330000007457838+0.021999998921133856j), (0.31199999971896164+0.5225999975901942j), (-0.8711999887924984+0.7919999822955295j)]


In [ ]:
class PEncoder:
    """An encoder for several complex numbers as specified in the CKKS scheme.

    Attributes:
        degree (int): Degree of polynomial that determines quotient ring.
        fft (FFTContext): FFTContext object to encode/decode.
    """

    def __init__(self, params):
        """Inits CKKSEncoder with the given parameters.

        Args:
            params (Parameters): Parameters including polynomial degree,
                plaintext modulus, and ciphertext modulus.
        """
        self.degree = params.poly_degree
        self.fft = FFTContext(self.degree * 2)

    def encode(self, values, scaling_factor):
        """Encodes complex numbers into a polynomial.

        Encodes an array of complex number into a polynomial.

        Args:
            values (list): List of complex numbers to encode.
            scaling_factor (float): Scaling factor to multiply by.

        Returns:
            A Plaintext object which represents the encoded value.
        """
        num_values = len(values)
        plain_len = num_values << 1

        # Canonical embedding inverse variant.
        to_scale = self.fft.embedding_inv(values)

        # Multiply by scaling factor, and split up real and imaginary parts.
        message = [0] * plain_len
        for i in range(num_values):
            message[i] = int(to_scale[i].real * scaling_factor + 0.5)
            message[i + num_values] = int(to_scale[i].imag * scaling_factor + 0.5)

        return Plaintext(Polynomial(plain_len, message), scaling_factor)


    def decode(self, plain):
        """Decodes a plaintext polynomial.

        Decodes a plaintext polynomial back to a list of integers.

        Args:
            plain (Plaintext): Plaintext to decode.

        Returns:
            A decoded list of integers.
        """
        if not isinstance(plain, Plaintext):
            raise ValueError("Input to decode must be a Plaintext")

        plain_len = len(plain.poly.coeffs)
        num_values = plain_len >> 1

        # Divide by scaling factor, and turn back into a complex number.
        message = [0] * num_values
        for i in range(num_values):
            message[i] = complex(plain.poly.coeffs[i] / plain.scaling_factor,
                                 plain.poly.coeffs[i + num_values] / plain.scaling_factor)

        # Compute canonical embedding variant.
        return self.fft.embedding(message)


In [23]:
import numpy as np
np.array([0.5, 0.3 + 0.2j, 0.78, 0.88j]) * (1 << 30)
np.fft.fft(np.arange(16), 8)

array([28.+0.j        , -4.+9.65685425j, -4.+4.j        , -4.+1.65685425j,
       -4.+0.j        , -4.-1.65685425j, -4.-4.j        , -4.-9.65685425j])

In [27]:
"""Example of CKKS multiplication."""

from ckks.ckks_encoder import CKKSEncoder


poly_degree = 8
ciph_modulus = 1 << 600
big_modulus = 1 << 1200
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor)
key_generator = CKKSKeyGenerator(params)

public_key = key_generator.public_key
secret_key = key_generator.secret_key
relin_key = key_generator.relin_key

encoder = CKKSEncoder(params)

message1 = [0.5, 0.3 + 0.2j, 0.78, 0.88j]
message2 = [0.2, 0.11, 0.4 + 0.67j, 0.9 + 0.99j]
plain1 = encoder.encode(message1, scaling_factor)
plain2 = encoder.encode(message2, scaling_factor)

print(plain1)


-68383068x^7 + -391013813x^6 + 24216163x^5 + 289910292x^4 + 115490928x^3 + -18981252x^2 + -268899682x + 424128020


In [30]:
from numpy.polynomial import Polynomial

class CKKSEncoder2:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, M: int):
        """Initialization of the encoder for M a power of 2. 
        
        xi, which is an M-th root of unity will, be used as a basis for our computations.
        """
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        
    @staticmethod
    def vandermonde(xi: np.complex128, M: int) -> np.array:
        """Computes the Vandermonde matrix from a m-th root of unity."""
        
        N = M //2
        matrix = []
        # We will generate each row of the matrix
        for i in range(N):
            # For each row we select a different root
            root = xi ** (2 * i + 1)
            row = []

            # Then we store its powers
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)
        return matrix
    
    def sigma_inverse(self, b: np.array) -> Polynomial:
        """Encodes the vector b in a polynomial using an M-th root of unity."""

        # First we create the Vandermonde matrix
        A = CKKSEncoder2.vandermonde(self.xi, self.M)

        # Then we solve the system
        coeffs = np.linalg.solve(A, b)

        # Finally we output the polynomial
        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by applying it to the M-th roots of unity."""

        outputs = []
        N = self.M //2

        # We simply apply the polynomial on the roots
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)

In [35]:
CKKSEncoder2(8).sigma_inverse([0.5, 0.3 + 0.2j, 0.78, 0.88j])

Polynomial([ 0.395     +0.27j      , -0.22273864+0.11667262j,
       -0.27      -0.245j     , -0.01767767-0.12374369j], domain=[-1.,  1.], window=[-1.,  1.], symbol='x')